### Simple example for a M/M/c queue using Simpy simulation environment.

At first, we need to make sure that the simpy module is available.

In [1]:
! pip install simpy

  Obtaining dependency information for simpy from https://files.pythonhosted.org/packages/48/72/920ed1224c94a8a5a69e6c1275ac7fe4eb911ba8feffddf469f1629d47f3/simpy-4.1.1-py3-none-any.whl.metadata


The source code has one class defined with three main methods:
* One defining the arrival process where the object waits for a random time until a new customer is put in line
* One defining a service process where an object is taken from the queue and the service time is again random
* One process which tracks the current queue length to allow for some analysis

Depending on the number of server, multiple services processes are started.


In [1]:
import simpy
import random
import matplotlib.pyplot as plt
import itertools
import numpy as np
from ipywidgets import interact, FloatSlider, IntSlider
import ipywidgets as widgets

#
# Simple implementation of an M/M/n model
#
class QueueSimulation:
    # Initialize the object
    def __init__(self, env, arrival_rate, service_rate, n_servers):
        self.env = env
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.n_servers = n_servers
        self.busy = False
        self.waiting_line = simpy.Store(env)
        self.waiting_times = []
        self.queue_length = []
        self.busy_times = []
        self.served_users = 0

    # Implements the arrival process to the queue
    def arrival(self):
        while True:                
            yield self.env.timeout(np.random.exponential(1.0 / self.arrival_rate))            
            yield self.waiting_line.put(self.env.now)               

    # Implements *one* server for the queue
    def server(self):
        while True:
            if len(self.waiting_line.items) > 0:
                self.busy = True
                entrance_time = yield self.waiting_line.get()                
                self.waiting_times.append((self.env.now - entrance_time))
                yield self.env.timeout(np.random.exponential(1.0 / self.service_rate))
                self.busy = False
                self.served_users = self.served_users + 1
            else:
                yield self.env.timeout(1)

    # Tracks the current status of the queue for logging purposes
    def tracker(self):
        while True:
            yield self.env.timeout(2)
            self.queue_length.append(len(self.waiting_line.items))
            if self.busy:
                self.busy_times.append(1)
            else:
                self.busy_times.append(0)

def run(arrival_rate, service_rate, n_servers):
    # Create the SimPy environment
    env = simpy.Environment()
    queue_simulation = QueueSimulation(env, arrival_rate, service_rate, n_servers)

    # Start the processes
    env.process(queue_simulation.arrival())
    env.process(queue_simulation.tracker())
    for i in range(n_servers):        
        env.process(queue_simulation.server())        

    # Run the simulation
    simulation_time = 10000
    env.run(until=simulation_time)

    # Plotting the results for queue length
    plt.figure(figsize=(8, 4))    
    plt.step(range(len(queue_simulation.queue_length)), queue_simulation.queue_length, where='post', label=f'Queue length')
    
    plt.xlabel('Time')
    plt.ylabel('Queue Length')
    plt.title(f'Queue Length Over Time for {n_servers:d} Service Processes (λ={arrival_rate:.2f}, μ={service_rate:.2f})')
    plt.legend()
    plt.show()    
    
    # Plotting the distribution of wait times
    plt.figure(figsize=(7, 3))
    plt.hist(queue_simulation.waiting_times, bins=30, alpha=0.7, edgecolor='black', cumulative=False)
    plt.xlabel('Wait Time')
    plt.ylabel('Number of Customers')
    plt.title('Distribution of Customer Wait Times')
    plt.show()

    # Output performance metrics
    print("System utilization: {:.2f}".format(np.sum(queue_simulation.busy_times) / len(queue_simulation.busy_times)))
    print("Throughput: {:.2f}".format(queue_simulation.served_users / simulation_time))
    print("Average queue length {:.2f}".format(np.average(queue_simulation.queue_length)))
    print("Average waiting times {:.2f}".format(np.average(queue_simulation.waiting_times)))
    
    return queue_simulation

# Interactive widgets to adjust parameters and rerun simulation
interact(run,         
         arrival_rate=FloatSlider(value=0.5, min=0.1, max=10.0, step=0.1, description='Arrival Rate (λ):'),
         service_rate=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Service Rate (μ):'),
        n_servers=IntSlider(value=2, min=1, max=10, step=1, description='Number of Service Processes:'))    

interactive(children=(FloatSlider(value=0.5, description='Arrival Rate (λ):', max=10.0, min=0.1), FloatSlider(…

<function __main__.run(arrival_rate, service_rate, n_servers)>